#Extract data from csv file

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('DOHMH_New_York_City_Restaurant_Inspection_Results.csv')
df = df.drop_duplicates()

clean_df = df[df['INSPECTION DATE'].apply(lambda x: x.split('/')[-1] in ('2016','2017','2018','2019','2020','2021'))]
clean_df.head()


,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,41350752,CHIPOTLE MEXICAN GRILL,Manhattan,25,WEST 45 STREET,10036.0,2123912081,Tex-Mex,03/05/2019,Violations were cited in the following area(s).,...,08/04/2021,Cycle Inspection / Initial Inspection,40.755847,-73.980516,105.0,4.0,9600.0,1034245.0,1.012610e+09,MN17
2,50078863,AMPLE HILLS CREAMERY,Queens,3402,30TH AVE,11103.0,9178640921,Frozen Desserts,09/06/2018,Violations were cited in the following area(s).,...,08/04/2021,Pre-permit (Operational) / Initial Inspection,40.765695,-73.918861,401.0,22.0,6300.0,4009084.0,4.006260e+09,QN70
3,41445602,FLOR DE MAYO,Manhattan,484,AMSTERDAM AVENUE,10024.0,2127873388,Latin American,04/20/2017,Violations were cited in the following area(s).,...,08/04/2021,Cycle Inspection / Re-inspection,40.785937,-73.976300,107.0,6.0,17100.0,1032780.0,1.012310e+09,MN12
4,50047667,LIBERTY BAGEL MIDTOWN,Manhattan,260,WEST 35 STREET,10001.0,2122791124,Bagels/Pretzels,07/27/2018,Violations were cited in the following area(s).,...,08/04/2021,Cycle Inspection / Initial Inspection,40.752337,-73.991785,105.0,3.0,10900.0,1014418.0,1.007840e+09,MN17
5,50079488,JOE & THE JUICE GREENWICH ST,Manhattan,295,GREENWICH STREET,10007.0,9179721892,Coffee/Tea,12/04/2019,Violations were cited in the following area(s).,...,08/04/2021,Administrative Miscellaneous / Initial Inspection,40.716002,-74.011017,101.0,1.0,2100.0,1066187.0,1.001378e+09,MN24


#Create restaurant dimension table

In [2]:
restaurant = clean_df[['DBA','BORO','BUILDING','STREET','ZIPCODE','CUISINE DESCRIPTION']].drop_duplicates(subset=['DBA'])
restaurant.rename(columns={'DBA':'Name','BUILDING':'Building','STREET':'Street','ZIPCODE':'Zipcode','CUISINE DESCRIPTION':'Cuisine'},inplace=True)
restaurant['RestaurantID'] = np.arange(1,restaurant.shape[0]+1)
restaurant.to_csv('Restaurant',index=False)
restaurant.head()

,Name,BORO,Building,Street,Zipcode,Cuisine,RestaurantID
0,CHIPOTLE MEXICAN GRILL,Manhattan,25,WEST 45 STREET,10036.0,Tex-Mex,1
2,AMPLE HILLS CREAMERY,Queens,3402,30TH AVE,11103.0,Frozen Desserts,2
3,FLOR DE MAYO,Manhattan,484,AMSTERDAM AVENUE,10024.0,Latin American,3
4,LIBERTY BAGEL MIDTOWN,Manhattan,260,WEST 35 STREET,10001.0,Bagels/Pretzels,4
5,JOE & THE JUICE GREENWICH ST,Manhattan,295,GREENWICH STREET,10007.0,Coffee/Tea,5


#Create violation dimension table

In [3]:
violation = clean_df[['VIOLATION CODE','VIOLATION DESCRIPTION','CRITICAL FLAG']].drop_duplicates(subset=['VIOLATION DESCRIPTION'])
violation.rename(columns={'VIOLATION CODE':'ViolationCode','VIOLATION DESCRIPTION':'ViolationDescription','CRITICAL FLAG':'IsViolationCritical'},inplace=True)

violation.sort_values(['ViolationCode'],inplace=True)
violation['ViolationID'] = np.arange(1,violation.shape[0]+1)

conditions = [
     violation['ViolationDescription'].str.contains('surface') == True,
     violation['ViolationDescription'].str.contains('Evidence') == True,
     violation['ViolationDescription'].str.contains('flies') == True,
     violation['ViolationDescription'].str.contains('º F|temperature') == True,
     violation['ViolationDescription'].str.contains('contaminat') == True,
     violation['ViolationDescription'].str.contains('roaches') == True,
     violation['ViolationDescription'].str.contains('Plumbing') == True,
     violation['ViolationDescription'].str.contains('sanit|Sanit') == True,
     violation['ViolationDescription'].str.contains('Certificate') == True,
     violation['ViolationDescription'].str.contains('sign not|Sign') == True,
     violation['ViolationDescription'].str.contains('wash') == True,
     violation['ViolationDescription'].str.contains('Pesticide') == True,
     violation['ViolationDescription'].str.contains('storage|Storage') == True,
     violation['ViolationDescription'].str.contains('re-served') == True,
     violation['ViolationDescription'].str.contains('hand') == True,
     violation['ViolationDescription'].str.contains('vermin') == True,
     violation['ViolationDescription'].str.contains('label|LABEL') == True,
     violation['ViolationDescription'].str.contains('thaw|Thaw') == True,
     violation['ViolationDescription'].str.contains('maintain') == True,
     violation['ViolationDescription'].str.contains('provided|policy|Document') == True,
     violation['ViolationDescription'].str.contains('smoking|Smoking') == True,
     violation['ViolationDescription'].str.contains('nutrition|calorie') == True,
     violation['ViolationDescription'].str.contains('canned|Canned') == True,
     violation['ViolationDescription'].str.contains('Lighting') == True,
     violation['ViolationDescription'].str.contains('polystyrene') == True,
     violation['ViolationDescription'].str.contains('Nuisance') == True,
     violation['ViolationDescription'].str.contains('cleanliness') == True,
     violation['ViolationDescription'].str.contains('Live animals') == True,
     violation['ViolationDescription'].str.contains('Sewage disposal') == True,
     violation['ViolationDescription'].str.contains('Single service item') == True
 ]
choices = ['Food Surface','Rodents','Flies','Temperature','Contamination','Roaches','Plumbing','Sanitation','Certificate','Sign Posting','Washing','Pesticide','Storage','Re-served Food','Hand Cleaning','Vermin','Label','Thawing','Maintenance','Compliance','Smoking','Nutrition Information','Canned Food','Lighting','Polystyrene','Nuisance','Cleanliness','Live Animals Other Than Fish','Sewage disposal','Single Service Item']
violation['ViolationMainType'] = np.select(conditions,choices,default='Other')

violation.to_csv('Violation',index=False)
violation.tail(10)

,ViolationCode,ViolationDescription,IsViolationCritical,ViolationID,ViolationMainType
1813,22E,ROP processing equipment not approved by DOHMH.,Not Critical,98,Other
229,22G,Sale or use of certain expanded polystyrene it...,Not Critical,99,Polystyrene
17,NaN,NaN,Not Applicable,100,Other
296,NaN,MISBRANDED AND LABELING,Not Critical,101,Label
610,NaN,"Bulb not shielded or shatterproof, in areas wh...",Not Critical,102,Temperature
1773,NaN,"A food containing artificial trans fat, with 0...",Not Critical,103,Other
8725,NaN,Failure to comply with an Order of the Board o...,Not Critical,104,Other
10271,NaN,Permit not conspicuously displayed.,Not Critical,105,Other
11110,NaN,"Document issued by the Board of Health, Commis...",Not Critical,106,Compliance
216943,NaN,Out of packaged sales of tobacco products,Not Critical,107,Other


#Create inspection dimention table

In [4]:
inspection = clean_df[['INSPECTION TYPE']].drop_duplicates(subset=['INSPECTION TYPE'])
inspection.rename(columns={'INSPECTION TYPE':'InspectionType'},inplace=True)
inspection['InspectionMainType'] = inspection['InspectionType'].apply(lambda x: str(x).split(' / ')[0])
inspection['InspectionSubType'] = inspection['InspectionType'].apply(lambda x: str(x).split(' / ')[1])
inspection['InspectionTypeID'] = np.arange(1,inspection.shape[0]+1)
inspection.to_csv('Inspection',index=False)
inspection.tail(10)


,InspectionType,InspectionMainType,InspectionSubType,InspectionTypeID
6551,Administrative Miscellaneous / Compliance Insp...,Administrative Miscellaneous,Compliance Inspection,22
7384,Trans Fat / Compliance Inspection,Trans Fat,Compliance Inspection,23
14894,Smoke-Free Air Act / Limited Inspection,Smoke-Free Air Act,Limited Inspection,24
23025,Calorie Posting / Compliance Inspection,Calorie Posting,Compliance Inspection,25
29600,Pre-permit (Non-operational) / Compliance Insp...,Pre-permit (Non-operational),Compliance Inspection,26
29888,Administrative Miscellaneous / Second Complian...,Administrative Miscellaneous,Second Compliance Inspection,27
51622,Cycle Inspection / Second Compliance Inspection,Cycle Inspection,Second Compliance Inspection,28
92278,Smoke-Free Air Act / Compliance Inspection,Smoke-Free Air Act,Compliance Inspection,29
205201,Trans Fat / Second Compliance Inspection,Trans Fat,Second Compliance Inspection,30
352215,Pre-permit (Non-operational) / Second Complian...,Pre-permit (Non-operational),Second Compliance Inspection,31


#Create time dimension table

In [5]:
def create_date_table(start='2016-01-01', end='2021-08-04'):
   df = pd.DataFrame({"date": pd.date_range(start, end)})
   df["DateOfWeek"] = df.date.dt.dayofweek
   df["DateOfMonth"] = df.date.dt.day
   df["DateOfYear"] = df.date.dt.dayofyear
   df["WeekOfYear"] = df.date.dt.isocalendar().week
   df["MonthNumber"] = df.date.dt.month
   df["MonthName"] = df.date.dt.month_name()
   df["YearNumber"] = df.date.dt.year
   df["IsWeekend"] = df["DateOfWeek"].apply(lambda x: True if (x > 5 or x == 0) else False)
   df.insert(0, 'DateID', (df.YearNumber.astype(str) + df.MonthNumber.astype(str).str.zfill(2) + df.DateOfMonth.astype(str).str.zfill(2)).astype(int))
   return df
date = create_date_table()
date.to_csv('Date',index=False)
date.head()

,DateID,date,DateOfWeek,DateOfMonth,DateOfYear,WeekOfYear,MonthNumber,MonthName,YearNumber,IsWeekend
0,20160101,2016-01-01,4,1,1,53,1,January,2016,False
1,20160102,2016-01-02,5,2,2,53,1,January,2016,False
2,20160103,2016-01-03,6,3,3,53,1,January,2016,True
3,20160104,2016-01-04,0,4,4,1,1,January,2016,True
4,20160105,2016-01-05,1,5,5,1,1,January,2016,False


#Create grade fact table

In [8]:
grade_restaurant = pd.merge(clean_df,restaurant,left_on='DBA',right_on='Name',how='inner').drop(['CAMIS','DBA','BORO_x','BUILDING','STREET','ZIPCODE','PHONE','CUISINE DESCRIPTION','ACTION','VIOLATION CODE','CRITICAL FLAG','RECORD DATE','GRADE DATE','Latitude','Longitude','Community Board','Council District','Census Tract','BIN','BBL','NTA','Name','BORO_y','Building','Street','Zipcode','Cuisine'],axis=1)
grade_inspection = pd.merge(grade_restaurant,inspection,left_on='INSPECTION TYPE',right_on='InspectionType',how='inner').drop(['INSPECTION TYPE','InspectionType','InspectionMainType','InspectionSubType'],axis=1)
grade_violation = pd.merge(grade_inspection,violation,left_on='VIOLATION DESCRIPTION',right_on='ViolationDescription',how='inner').drop(['ViolationDescription','ViolationCode','VIOLATION DESCRIPTION','IsViolationCritical','ViolationMainType'],axis=1)
grade_violation['INSPECTION DATE'] = pd.to_datetime(grade_violation['INSPECTION DATE'])
grade_fact = pd.merge(grade_violation,date,left_on='INSPECTION DATE',right_on='date',how='inner').drop(['INSPECTION DATE','date','DateOfWeek','DateOfMonth','DateOfYear','WeekOfYear','MonthNumber','MonthName','YearNumber','IsWeekend'],axis=1)
grade_fact.rename(columns={'DateID':'InspectionDate','SCORE': 'Score','GRADE':'Grade'},inplace=True)

grade_fact['FactGradeID']=np.arange(1,grade_fact.shape[0]+1)
grade_fact.to_csv('FactGrade',index=False)

grade_fact.tail(20)


,Score,Grade,RestaurantID,InspectionTypeID,ViolationID,InspectionDate,FactGradeID
377083,NaN,NaN,14834,4,96,20210421,377084
377084,NaN,NaN,15984,4,96,20210222,377085
377085,NaN,NaN,17272,4,96,20210208,377086
377086,NaN,NaN,8529,4,102,20210208,377087
377087,NaN,NaN,18466,4,96,20210506,377088
377088,NaN,NaN,19103,4,96,20210430,377089
377089,NaN,N,19974,4,96,20210525,377090
377090,NaN,N,19974,5,79,20210525,377091
377091,NaN,N,24,4,102,20210303,377092
377092,NaN,NaN,11207,4,102,20210527,377093


#Create visit fact table

In [9]:
visit_restaurant = pd.merge(clean_df,restaurant,left_on='DBA',right_on='Name',how='inner').drop(['CAMIS','DBA','BORO_x','BUILDING','STREET','ZIPCODE','PHONE','CUISINE DESCRIPTION','CRITICAL FLAG','SCORE','GRADE','GRADE DATE','RECORD DATE','Latitude','Longitude','Community Board','Council District','Census Tract','BIN','BBL','NTA','Name','BORO_y','Building','Street','Zipcode','Cuisine'],axis=1)
visit_violation = pd.merge(visit_restaurant,violation,left_on='VIOLATION DESCRIPTION',right_on='ViolationDescription',how='inner').drop(['VIOLATION CODE','ViolationCode','VIOLATION DESCRIPTION','ViolationDescription','IsViolationCritical','ViolationMainType'],axis=1)
visit_violation['VisitHadViolation'] = visit_violation['ACTION'].apply(lambda x: False if x == 'No violations were recorded at the time of this inspection.' else True)
visit_violation['VisitViolationCount'] = visit_violation[visit_violation['VisitHadViolation'] == True].groupby(['RestaurantID','INSPECTION DATE'])['VisitHadViolation'].transform('count')
visit_violation['VisitViolationCount'] = visit_violation['VisitViolationCount'].fillna(0).astype(int)
visit = pd.merge(visit_violation,inspection,left_on='INSPECTION TYPE',right_on='InspectionType',how='inner').drop(['INSPECTION TYPE','InspectionType','InspectionMainType','InspectionSubType'],axis=1)
visit['INSPECTION DATE'] = pd.to_datetime(visit['INSPECTION DATE'])
visit['VisitSequence'] = visit.groupby('RestaurantID')['INSPECTION DATE'].rank().astype(int)
visit.sort_values('INSPECTION DATE')
visit['DateLagged'] = visit.sort_values(['INSPECTION DATE','RestaurantID']).groupby('RestaurantID')['INSPECTION DATE'].shift(1)
visit['VisitDaysBetween'] = (visit['INSPECTION DATE'] - visit['DateLagged']).dt.days
visit['VisitDaysBetween'] = visit['VisitDaysBetween'].fillna(0).astype(int)
visit_fact = pd.merge(visit,date,left_on='INSPECTION DATE',right_on='date',how='inner').drop(['INSPECTION DATE','date','DateOfWeek','DateOfMonth','DateOfYear','WeekOfYear','MonthNumber','MonthName','YearNumber','IsWeekend','DateLagged'],axis=1)
visit_fact.rename(columns={'DateID':'InspectionDate','ACTION':'Action'},inplace=True)

visit_fact['FactVisitID']=np.arange(1,visit_fact.shape[0]+1)
visit_fact.to_csv('FactVisit',index=False)
visit_fact.head(20)



,Action,RestaurantID,ViolationID,VisitHadViolation,VisitViolationCount,InspectionTypeID,VisitSequence,VisitDaysBetween,InspectionDate,FactVisitID
0,Violations were cited in the following area(s).,1,60,True,2,1,437,6,20190305,1
1,Violations were cited in the following area(s).,1,60,True,2,1,437,0,20190305,2
2,Violations were cited in the following area(s).,40,60,True,2,1,1300,4,20190305,3
3,Violations were cited in the following area(s).,47,60,True,3,1,2772,1,20190305,4
4,Violations were cited in the following area(s).,162,60,True,4,1,10,216,20190305,5
5,Violations were cited in the following area(s).,244,60,True,4,1,130,4,20190305,6
6,Violations were cited in the following area(s).,277,60,True,4,1,204,1,20190305,7
7,Violations were cited in the following area(s).,326,60,True,3,1,65,35,20190305,8
8,Violations were cited in the following area(s).,335,60,True,4,1,21,355,20190305,9
9,Violations were cited in the following area(s).,350,60,True,6,1,583,1,20190305,10
